In [17]:
import pandas as pd
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn import metrics, svm, tree, ensemble, linear_model, naive_bayes
from textblob.classifiers import NaiveBayesClassifier

import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer
from sklearn import *
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold

/Users/yashikagoyal/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/yashikagoyal/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/yashikagoyal/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module wi

In [2]:
dataTrain = pd.read_csv('data-2_train.csv')

In [3]:
dataTrain.head()

,example_id,text,aspect_term,term_location,class
0,3121_0,But the staff was so horrible to us.,staff,8--13,-1
1,2777_0,To be completely fair[comma] the only redeemin...,food,57--61,1
2,1634_0,The food is uniformly exceptional[comma] with ...,food,4--8,1
3,1634_1,The food is uniformly exceptional[comma] with ...,kitchen,55--62,1
4,1634_2,The food is uniformly exceptional[comma] with ...,menu,141--145,0


In [4]:
dataTrain.columns = ['example_id','text','aspect_term','term_location', 'class']

In [5]:
text = dataTrain['text']
rClass = dataTrain['class']


In [6]:
# testimonial = TextBlob(dataTrain['text'][0])
# testimonial.sentiment
# testimonial.sentiment.polarity


In [7]:
# X_train, X_test, y_train, y_test = train_test_split(text, rClass, test_size=0.2)

In [8]:
def splitSentences(text, aspect):
    conjuctionWords = {'any', 'also', 'but', 'that' , 'hence', 'therefore', 'if', 'only', 'so'}
    pos = 0
    a_pos = 0
    array = []
    text = text.split()
#     print(aspect)
    for index,t in enumerate(text):
        
        if t in conjuctionWords:
#             print(t)
            pos = index
        if t == aspect:
#             print(t)
            a_pos = index
   
    if pos < a_pos:
        text = text[pos+1:a_pos+len(aspect)+1]
    else:
        text = text[:a_pos + len(aspect) + 1]
        
    return text

In [9]:
text = dataTrain['text']
# text = posTag(text)
rClass = dataTrain['class']
aspect = dataTrain['aspect_term']
# print(text[1])
for i in range(len(text)):
    temp= splitSentences(text[i], aspect[i])
    text[i] = ' '.join(temp)
#     print(type(text[i]))
    

/Users/yashikagoyal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
# For pos Tagging
def posTag(text1):
    text = list()
    for item in text1:
        item = re.findall(r'\w+', item)
        item = nltk.pos_tag(item)
        item1 = list()

        lemmatizer = WordNetLemmatizer()
        for index, g in enumerate(item):
            if g[1] == 'NN' or g[1] == 'NNS' or g[1] == 'NNP' or g[1] == 'NNPS' or g[1] == 'JJ' or g[1] == 'JJR' or g[1] == 'JJS' or g[1] == 'IN':
                item2 = lemmatizer.lemmatize(g[0])
                item1.append(item2)

        text.append(" ".join(item1))
    return text

In [19]:
# text=text.replace("comma","",regex=True)
# text=text.replace("\[]","",regex=True)
# text = [t.lower() for t in text]
text = posTag(text)


AttributeError: 'list' object has no attribute 'replace'

In [13]:
sentiment = []
for review in text:
    testimonial = TextBlob(review)
    p = testimonial.sentiment.polarity
    
    if p >= 0.225:
        sentiment.append(1)
    elif p <= -0.225:
        sentiment.append(-1)
    else:
        sentiment.append(0)
    

In [25]:
accScore = metrics.accuracy_score(rClass,sentiment)
print('Accuracy: ', accScore,'\n')
lbl = [1,0,-1]
f1Score = metrics.f1_score(rClass,sentiment,average=None,labels=lbl)
precision = metrics.precision_score(rClass,sentiment,average=None,labels=lbl)
recall = metrics.recall_score(rClass,sentiment,average=None,labels=lbl)
f1=0
p = 0;
r=0;
for i in range(len(lbl)):
    print("F1-Score of %s class: %f" %(lbl[i],f1Score[i]))
    print("Recall of %s class: %f" %(lbl[i],recall[i]))
    print("Precision of %s class: %f" %(lbl[i],precision[i]))
    print()
    f1 = f1 + f1Score[i]
    p = p + precision[i]
    r = r + recall[i]
print("F1-Score of class: %f" %(f1/3))
print("Recall of class: %f" %(r/3))
print("Precision of class: %f" %(p/3))

Accuracy:  0.5124930594114381 

F1-Score of 1 class: 0.674238
Recall of 1 class: 0.562384
Precision of 1 class: 0.841632

F1-Score of 0 class: 0.368794
Recall of 0 class: 0.739336
Precision of 0 class: 0.245669

F1-Score of -1 class: 0.304924
Recall of -1 class: 0.200000
Precision of -1 class: 0.641434

F1-Score of class: 0.449319
Recall of class: 0.500574
Precision of class: 0.576245
